1. Read json file
2. Set up the sampler through batchrunner
3. generate the input file
4. save it.

In [ ]:
# Import relevant libraries
import os
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT

In [ ]:
os.getcwd()

In [ ]:
# Define variables
param_path = "./parameters_01.json"

n_samples = 64 # Powers of 2

simulation_out_path = f'outputs/simulations/output_{n_samples}_samples/'

In [ ]:
# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here? 
br.setup_sampler(param_path)
br.sample(n_samples)

In [ ]:
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [ ]:
br.map_vessel_volume()

In [ ]:
br.samples

In [ ]:
# Save the samples to a CSV file
output_path = "./samples.csv"

br.samples.to_csv(output_path, index=False)

In [ ]:
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                   time_setup=TEMPLATE_TIME_SETUP_DICT)

In [ ]:
os.makedirs(simulation_out_path, exist_ok=True)

In [ ]:
simulations = br.run_batch(
            n_jobs=10,
            output_path=simulation_out_path,
        )

In [ ]:
## Setting up the headers of the columns for my simulation summary similar to fay
headers = [
    'v_ao', 'v_art', 'v_ven', 'v_la', 'v_lv', 'p_lv', 'q_av', 'p_ao', 'p_art',
    'q_ao', 'p_ven', 'q_art', 'p_la', 'q_ven', 'q_mv', 'T'
]

summary_rows = []

for i_sim in range(len(simulations)): ## stuck with the length of simulations like fay
    sim_df = simulations[i_sim]
    
    # Skip if not a DataFrame this was needed because sometimes simulations can return None or other types. or false.
    if not isinstance(sim_df, pd.DataFrame):
        print(f"Simulation {i_sim} is not a DataFrame, skipping.")
        continue

    row = {} ##the 'dictiionary' to hold the summary for each simulation
    for h in headers: ## using the headers defined above to make the new csv file headers eg v_ao_mean, v_ao_max, etc.
        row[f"{h}_mean"] = sim_df[h].mean()
        row[f"{h}_max"] = sim_df[h].max()
    summary_rows.append(row) # this is a list that is being used to collect summary data for our simulations.
    print(f"Simulation {i_sim} done.") # simple way to track progress 

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv('simulation_summary.csv', index=False)
print("Saved as simulation_summary.csv") # not really needed but nice to show the file has been saved! :D